In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

In [2]:
# set the path to the image folder
path = "/kaggle/input/emotion-detection-fer/train"

In [3]:
# define image size and batch size
img_size = (48,48)
batch_size = 32

In [4]:
# create data generator for training images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [5]:
train_generator = train_datagen.flow_from_directory(
    directory=path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # use the training subset of the data
)

Found 22968 images belonging to 7 classes.


In [6]:
# create data generator for validation images
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # split the data into training and validation sets
)

In [7]:
val_generator = val_datagen.flow_from_directory(
    directory=path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # use the validation subset of the data
)

Found 5741 images belonging to 7 classes.


In [8]:
# load the pre-trained ResNet50 model
resnet = ResNet50(input_shape=(48,48, 3), weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 0s 0us/step


In [9]:
# freeze the pre-trained layers
for layer in resnet.layers:
    layer.trainable = False

# add custom layers on top of the pre-trained model
x = Flatten()(resnet.output)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(7, activation='softmax')(x)

In [10]:
# create the final model
model = Model(inputs=resnet.input, outputs=x)

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
# train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50
)

Epoch 1/50
718/718 [==============================] - 281s 371ms/step - loss: 1.8092 - accuracy: 0.2471 - val_loss: 1.7923 - val_accuracy: 0.2547
Epoch 2/50
718/718 [==============================] - 68s 94ms/step - loss: 1.7823 - accuracy: 0.2603 - val_loss: 1.7638 - val_accuracy: 0.2822
Epoch 3/50
718/718 [==============================] - 74s 103ms/step - loss: 1.7591 - accuracy: 0.2738 - val_loss: 1.7664 - val_accuracy: 0.2667
Epoch 4/50
718/718 [==============================] - 68s 94ms/step - loss: 1.7485 - accuracy: 0.2796 - val_loss: 1.7661 - val_accuracy: 0.2733
Epoch 5/50
718/718 [==============================] - 68s 95ms/step - loss: 1.7392 - accuracy: 0.2850 - val_loss: 1.7716 - val_accuracy: 0.2792
Epoch 6/50
718/718 [==============================] - 140s 195ms/step - loss: 1.7334 - accuracy: 0.2878 - val_loss: 1.7025 - val_accuracy: 0.3066
Epoch 7/50
718/718 [==============================] - 69s 97ms/step - loss: 1.7121 - accuracy: 0.2995 - val_loss: 1.7068 - val_accu